# Mean variance optimization set up example

Frans de Ruiter, CQM
For assignment and data, see https://www.fransderuiter.com/JADS/
***

### Setup
Make sure you have installed all the packages listed below, as well as the ipopt solver (see installation instructions from previous class).

In [ ]:
import numpy as np
import pandas as pd
from pyomo.environ import *
import matplotlib.pyplot as plt
from matplotlib import cm

### Read data

In [ ]:
data_path = "https://www.fransderuiter.com/JADS/Meanvariance/MeanVarPortfolio.xlsx"
# Create pandas table
df_input = pd.read_excel(data_path, sheet_name=0, header=1, skiprows=4, index_col=0)

# Show table
df_input.head()

In [ ]:
## Rename the first two columns

newcolumn_values = df_input.columns.values
newcolumn_values[0] = 'Mean'
newcolumn_values[1] = 'Variance'

df_input.columns = newcolumn_values

# Show table with removed column
df_input.head()

### Model parameters

In [ ]:
# Set with assets (indices)
assets = df_input.columns[2:] # first two columns are only mean and variance

# risk parameter
alpha = 0.012

### Plot means and variances of portfolios

In [ ]:
# Initialize plot
fig=plt.figure(figsize=(16,16))

# Plot mean returns and variances of individual assets
plt.scatter([df_input.loc[i,'Variance'] for i in assets], 
            [100*df_input.loc[i,'Mean'] for i in assets], 
            s = 200*np.ones(assets.size),
            alpha=0.9)

plt.title("Mean-Variance portfolios")
plt.xlabel("Variance")
plt.ylabel("Mean return (\%)")
plt.show()

## Model implementation

In [ ]:
# Tips:
# acces the mean of asset
i = 'ME1 INV2'
# by:
return_example = df_input.loc[i,'Mean']

# acces the covariance of two assets by
i = 'ME1 INV2'
j = 'ME1 INV3'

variance_example = df_input.loc[i,j]

In [ ]:
## TODO: fill in the dots

def MeanVarModelConstruction(alpha,df_input):
    # Create model
    m = ConcreteModel()

    # TODO: Add variables, objective and constraints to the model. 
    # In the objective you have to specify the direction (sense): is this a minimization or maximization problem?
    
    # Variables
    m.amount = Var(...) # note that short sellings is not allowed, what does that mean for the range of values that he variable can take?

    # Objective
    m.value = Objective(..., sense =...)

    # Constraints on budget
    m.budget = Constraint(....)

    return m

## Function to solve the model

In [ ]:
def OptimizeMeanVarModel(m,printResults,showprogress = False):
    # Optimize
    solver = SolverFactory('ipopt') # Take the ipopt solver for nonlinear problems
    status = solver.solve(m,tee=showprogress,)

    # Print the status of the solved model
    
    if printResults:
        print("Risk aversion parameter = %s \n" % alpha)
        print("Status = %s \n" % status.solver.termination_condition)
    
    obj_opt = value(m.value)
    mean_opt = sum([value(m.amount[i]*df_input.loc[i,'Mean']) for i in assets])
    variance_opt = sum(df_input.loc[i,j]*value(m.amount[i])*value(m.amount[j]) for i in assets for j in assets)
    
    return (obj_opt, mean_opt, variance_opt)

## Plot and show the solution

In [ ]:
# Construct model with chosen alpha
m = MeanVarModelConstruction(alpha,df_input)
# Optimize
obj_opt, mean_opt, variance_opt = OptimizeMeanVarModel(m,True,False)

print("Objective value model \t= %.3f" % obj_opt)
print("\t Mean return \t= %.3f" % mean_opt)
print("\t Variance \t= %.3f\n" % variance_opt)

print("Assets invested in (and amount):")
print("Portfolio composition:")
for i in assets:
    if value(m.amount[i] > 0.0001):
        print("\t"+ i + " : \t" + str(value(m.amount[i]))) 

## Plot the solution

In [ ]:
# Initialize plot
fig=plt.figure(figsize=(16,16))

# Plot mean returns and variances of individual assets
plt.scatter([df_input.loc[i,'Variance'] for i in assets], 
            [100*df_input.loc[i,'Mean'] for i in assets], 
            s = 200*np.ones(assets.size),
            alpha=0.9)
# Plot mean returns and variances of new optimal portfolio
plt.scatter(variance_opt, 
            100*mean_opt, 
            s = 200*np.ones(assets.size),
            c = 'red',
            alpha=0.9)

plt.title("Mean-Variance portfolios")
plt.xlabel("Variance")
plt.ylabel("Mean return (\%)")
plt.show()

## Bonus question:
Suppose we allow for short selling, short position cannot be more than 10% of your long position. How can we incorporate this in our model?

Hint 1: First try to write this new constraint on paper before you try to implement it.

Hint 2: Introduce two new parameters $x_long \geq 0$ and $x_short \geq 0$ that indicate the short and long position. So the total position is $x = x_long - x_short$. How do you incorporate the 10% requirement constraint?

In [ ]:
# TODO:
# 1) write down the new constraint on paper.
# 2) Make new optimization model for the mean-variance problem that allows for short-selling.